In [45]:
# Import libraries
import torch as th
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm.auto import trange
from sklearn.model_selection import train_test_split
import numpy as np

In [61]:
# Import datasets

sep_data = th.load('../datasets/sep_states.pt')

ent_data = th.load('../datasets/ent_states.pt')

sep_data_pair = th.cat((sep_data.real, sep_data.imag), dim=0)
ent_data_pair = th.stack((ent_data.real, ent_data.imag), dim=-1)

sep_data.shape, sep_data_pair.shape

(torch.Size([10000, 4]), torch.Size([20000, 4]))

In [59]:
t1 = th.tensor([1, 2, 3])
t2 = th.tensor([4, 5, 6])

t3 = th.stack((t1, t2), dim=-1)
t4 = th.cat((t1, t2), dim=0)

t3, t4

(tensor([[1, 4],
         [2, 5],
         [3, 6]]),
 tensor([1, 2, 3, 4, 5, 6]))

In [58]:
mat1 = th.tensor(([1, 2], [3, 4]))
mat1, mat1.flatten(start_dim=0)

(tensor([[1, 2],
         [3, 4]]),
 tensor([1, 2, 3, 4]))

In [31]:
sep_data_pair = sep_data_pair.reshape(sep_data_pair.shape[0], -1)

In [ ]:
BATCH_SIZE = 64
ent_loader = DataLoader(sep_data_pair.reshape(-1, 1, 4, 2), batch_size=BATCH_SIZE, shuffle=True)
sep_loader = DataLoader(ent_data_pair.reshape(-1, 1, 4, 2), batch_size=BATCH_SIZE, shuffle=True)

In [16]:
# Define the loss function

def custom_loss(x, x_hat, mean, logvar):
    reproduction_loss = nn.functional.binary_cross_entropy(x_hat, x, reduction='sum')
    KLD = -0.5 * th.sum(1 + logvar - mean * mean - logvar.exp())
    
    return reproduction_loss + KLD

In [21]:
class VAE_fc(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(VAE_fc, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size[0])
        self.fc2 = nn.Linear(hidden_size[0], hidden_size[1])
        
        self.leaky_relu = nn.LeakyReLU(0.2)
        
        self.fc3 = nn.Linear(hidden_size[1], hidden_size[0])
        self.fc4 = nn.Linear(hidden_size[0], input_size)
    
    def encode(self, x):
        # print("flatten", x.shape)
        x = self.fc1(x)
        x = self.leaky_relu(x)
        # print("fc1", x.shape)
        
        x = self.fc2(x)
        x = self.leaky_relu(x)
        # print("fc2", x.shape)
        return x
    
    def decode(self, x):
        x = self.fc3(x)
        x = self.leaky_relu(x)
        
        x = self.fc4(x)
        x = self.leaky_relu(x)
        return nn.Sigmoid()(x)
    
    def forward(self, x):
        encoded = self.encode(x)
        decoded = self.decode(encoded)
        return encoded, decoded

model = VAE_fc(input_size=4 * 2, hidden_size=[32, 16])
for i, data in enumerate(ent_loader):
    model(data)
    break

In [27]:
def train_model(model, train_loader, epochs, optimizer, criterion, device):
    trained_model = model.to(device)
    trained_model.train()
    for epoch in range(epochs):
        total_loss = 0.0
        for _, data in enumerate(train_loader):
            data = data.to(device)
            _, decoded = trained_model(data)
            loss = criterion(data, decoded)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * data.size(0)
        epoch_loss = total_loss / len(train_loader.dataset)
        
        print('Epoch: {}, Loss: {:.4f}'.format(epoch, epoch_loss))
    return trained_model

In [28]:
model = VAE_fc(input_size=4 * 2, hidden_size=[32, 16])
optimizer = th.optim.Adam(model.parameters(), lr=0.01)
model = train_model(model=model,
                    train_loader=ent_loader,
                    epochs=5,
                    optimizer=optimizer,
                    criterion=nn.MSELoss(),
                    device='cpu')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([64, 8])) that is different to the input size (torch.Size([64, 1, 4, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (2) must match the size of tensor b (8) at non-singleton dimension 3